<a href="https://colab.research.google.com/github/LorenzoCorbinelli/MLSA-project/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load the model
directory = '/content/drive/MyDrive/Magistrale/Machine learning for software analysis/MLSA/Model'

model = RobertaForMaskedLM.from_pretrained(directory)
tokenizer = RobertaTokenizer.from_pretrained(directory)

In [ ]:
code_example = "def is_zero(x):\n return x==<mask>"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(code_example)
print(outputs)

In [ ]:
code_example = "def add(a, b):\n return a+<mask>"
outputs = fill_mask(code_example)
print(outputs)

In [ ]:
code_example = "def add(a, b):\n return a<mask>"
outputs = fill_mask(code_example)
print(outputs)

In [ ]:
code_example = "for element <mask>"
outputs = fill_mask(code_example)
print(outputs)